# Libraries

In [45]:
import numpy as np
import pandas as pd
from time import perf_counter
import matplotlib.pyplot as plt

# Excercise 1

In [43]:
def A1(float_type, n):
    A = np.array([[1 / (i + j - 1) if i != 1 else 1 
        for j in range(1, n + 1)] 
        for i in range(1, n + 1)]).astype(float_type)
    return A

def gauss(A, B):
    n = np.shape(A)[0]
    C = np.hstack([A, B.reshape((n, 1))]).astype(np.float64)

    for i in range(n):
        for j in range(i + 1, n):
            ratio = C[j][i] / C[i][i]
            C[j] = C[j] - ratio * C[i]

    X = C[:, n]
    X[n - 1] /= C[n - 1][n - 1]
    for i in range(n - 2, -1, -1):
        X[i] -= np.sum(C[i][i + 1:n] * X[i + 1:n])
        X[i] /= C[i][i]
    return X

def ex1():
    result = []
    ns = [2, 3, 4, 5, 6, 7, 10, 15, 20, 50, 100, 150, 200, 300]
    float_types = [np.float32, np.float64]

    for n in ns:
        for ft in float_types:
            A = A1(ft, n)
            X_vec = np.array([1 if i % 2 == 0 else -1 for i in range(n)])
            B = np.matmul(A, X_vec)
            X = gauss_(A, B)
            norm = np.linalg.norm(X_vec - X)
            result += [norm]
    df = pd.DataFrame(data={"n": ns,
                            "float32": result[::2],
                            "float64": result[1::2]})
        
    return df

datas = ex1()
datas

,n,float32,float64
0,2,0.000000e+00,0.000000e+00
1,3,0.000000e+00,0.000000e+00
2,4,6.646519e-15,3.018715e-13
3,5,7.282637e-13,9.229383e-12
4,6,5.440998e-11,3.637978e-10
5,7,3.695169e-09,1.360925e-08
6,10,6.730517e-09,1.662034e-04
7,15,2.391005e-08,1.504852e+01
8,20,1.779323e-08,8.714011e+02
9,50,1.471070e-07,2.460859e+02


# Excercise 2

In [38]:
def A2(float_type, n):
    A = np.zeros((n, n)).astype(float_type)
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            if j >= i:
                A[i - 1][j - 1] = 2 * i / j
            else:
                A[i - 1][j - 1] = A[j - 1][i - 1]

    return A

def ex2():
    result = []
    ns = [2, 3, 4, 5, 6, 7, 10, 15, 20, 50, 100, 150, 200, 300]
    float_types = [np.float32, np.float64]

    for n in ns:
        for ft in float_types:
            A = A2(ft, n)
            X_vec = np.array([1 if i % 2 == 0 else -1 for i in range(n)])
            B = np.matmul(A, X_vec)
            X = gauss(A, B)
            norm = np.linalg.norm(X_vec - X)
            result += [norm]
    df = pd.DataFrame(data={"n": ns,
                            "float32": result[::2],
                            "float64": result[1::2]})
        
    return df

datas = ex2()
datas

,n,float32,float64
0,2,0.000000e+00,0.000000e+00
1,3,0.000000e+00,3.140185e-16
2,4,0.000000e+00,2.482534e-16
3,5,2.482534e-16,4.154074e-16
4,6,3.140185e-16,9.742168e-16
5,7,2.482534e-16,1.694682e-15
6,10,2.991428e-15,3.082744e-15
7,15,3.612919e-15,2.836115e-14
8,20,1.971889e-14,3.809004e-14
9,50,1.362741e-13,3.460566e-13


# Conditionality indicator

In [41]:
def A1_(n):
    return np.array([[1 / (i + j - 1) if i != 1 else 1 
                      for j in range(1, n + 1)] 
                      for i in range(1, n + 1)])

def A2_(n):
    A = np.zeros((n, n))
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            if j >= i:
                A[i - 1][j - 1] = 2 * i / j
            else:
                A[i - 1][j - 1] = A[j - 1][i - 1]
    return A

def norm(A):
    n = len(A)
    return max(sum(A[i][j] for j in range(n)) for i in range(n))

def conditioning_factor(A):
    A_inv = np.linalg.inv(A)
    return norm(A_inv) * norm(A)

def condition_number():
    ns = [2, 3, 4, 5, 6, 7, 10, 15, 20, 50, 100, 150, 200, 300]
    result = []
    for n in ns:
        con_num_1 = conditioning_factor(A1_(n))
        con_num_2 = conditioning_factor(A2_(n))
        result += [con_num_1, con_num_2]
    df = pd.DataFrame(data={"n":ns,
                            "ex 1 condition number":result[::2],
                            "ex 2 condition number":result[1::2]})
    return df

condition_df = condition_number()
condition_df


,n,ex 1 condition number,ex 2 condition number
0,2,8.000000e+00,1.000000
1,3,2.160000e+02,1.444444
2,4,2.880000e+03,1.833333
3,5,2.800000e+04,2.233333
4,6,2.268000e+05,2.644444
5,7,1.629936e+06,3.031746
6,10,8.841438e+08,4.249206
7,15,1.733309e+11,6.268898
8,20,4.003898e+11,8.289565
9,50,1.764101e+12,20.420510


# Excercise 3

In [46]:
def A3(n):
    A = np.zeros((n, n))
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            if i == j:
                A[i - 1][j - 1] = -3 * i - 7
            elif j == i + 1:
                A[i - 1][j - 1] = i
            elif i > j == i - 1:
                A[i - 1][j - 1] = 3 / i

    return A

def thomas(A, B):
    n = np.shape(A)[0]
    C = np.zeros(n)
    C[0] = A[0][0]

    X = np.zeros(n)
    X[0] = B[0]

    for i in range(1, n):
        ratio = A[i][i - 1] / C[i - 1]
        C[i] = A[i][i] - ratio * A[i - 1][i]
        X[i] = B[i] - ratio * X[i - 1]

    X[n - 1] = X[n - 1] / C[n - 1]
    for i in range(n - 2, -1, -1):
        X[i] = (X[i] - A[i][i + 1] * X[i + 1]) / C[i]

    return X    

def ex3():
    result = []
    ns = [2, 3, 4, 5, 6, 7, 10, 15, 20, 50, 100, 150, 200, 300]

    for n in ns:
        A = A3(n)
        X_vec = np.array([1 if i % 2 == 0 else -1 for i in range(n)])
        B = np.matmul(A, X_vec)
        g_start = perf_counter()
        X_g = gauss(A, B)
        g_end = perf_counter()
        g_time = g_end - g_start
        err_g = np.linalg.norm(X_vec - X_g)

        t_start = perf_counter()
        X_t = thomas(A, B)
        t_end = perf_counter()
        t_time = t_end - t_start
        err_t = np.linalg.norm(X_vec - X_t)

        result += [err_g, err_t, g_time, t_time]

    df = pd.DataFrame(data={"n": ns,
                            "gauss": result[::4],
                            "thomas": result[1::4],
                            "gauss time [s]": result[2::4],
                            "thomas time [s]": result[3::4]})
    return df

datas = ex3()
datas

,n,gauss,thomas,gauss time [s],thomas time [s]
0,2,0.000000e+00,0.000000e+00,0.000151,0.000038
1,3,0.000000e+00,0.000000e+00,0.000105,0.000014
2,4,0.000000e+00,0.000000e+00,0.000078,0.000016
3,5,0.000000e+00,0.000000e+00,0.000104,0.000017
4,6,0.000000e+00,0.000000e+00,0.000133,0.000021
5,7,0.000000e+00,0.000000e+00,0.000169,0.000024
6,10,2.482534e-16,2.482534e-16,0.000393,0.000033
7,15,2.482534e-16,2.482534e-16,0.000628,0.000046
8,20,4.839350e-16,4.839350e-16,0.001143,0.000061
9,50,8.158440e-16,8.158440e-16,0.007279,0.000171
